In [1]:
import pandas as pd
from datetime import date, timedelta
import folium
from folium import Marker
from folium.plugins import MarkerCluster, HeatMap
import math
import time
import webbrowser

In [11]:
populationData = pd.read_csv('../Data/populationbycountry2021lat_long.csv')

In [12]:
populationData.head()

,Entity,Population,latitude,longitude
0,Afghanistan,38913707,33.939110,67.709953
1,Albania,2944804,41.153332,20.168331
2,Algeria,43969303,28.033886,1.659626
3,Andorra,77276,42.546245,1.601554
4,Angola,33894484,-11.202692,17.873887


In [13]:
populationData.shape

(187, 4)

In [14]:
# Get the most recent date for filtering
freshDate = date.today() - timedelta(days=2)
freshDate = date.strftime(freshDate,"%Y%m%d")
freshDate = freshDate[0:4] + "-" + freshDate[4:6] + "-" + freshDate[6:8]

In [20]:
# Vaccination data, for most recent date
vaccinationData = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv')
vaccinationByLocation = vaccinationData.loc[(vaccinationData.date == freshDate)][["location", "people_vaccinated"]].reset_index(drop=True)

In [21]:
vaccinationByLocation.head()

,location,people_vaccinated
0,Argentina,481689.0
1,Australia,2879.0
2,Austria,312624.0
3,Bahrain,281425.0
4,Bangladesh,2491053.0


In [22]:
vaccinationByLocation.shape

(71, 2)

In [23]:
# Vaccination and population data
vaccinationAndPopulationByLocation = pd.merge(populationData, vaccinationByLocation, left_on='Entity',right_on='location').drop(columns="location").dropna(axis=0).reset_index(drop=True)

In [25]:
# Calculate percentage vaccinated by state
vaccinationAndPopulationByLocation["percent_vaccinated"] = vaccinationAndPopulationByLocation["people_vaccinated"] / vaccinationAndPopulationByLocation["Population"]

In [26]:
vaccinationAndPopulationByLocation

,Entity,Population,latitude,longitude,people_vaccinated,percent_vaccinated
0,Argentina,45914426,-38.416097,-63.616672,481689.0,0.010491
1,Australia,25700455,-25.274398,133.775136,2879.0,0.000112
2,Austria,8800683,47.516231,14.550072,312624.0,0.035523
3,Bahrain,1744295,25.930414,50.637772,281425.0,0.161340
4,Bangladesh,171499635,23.684994,90.356331,2491053.0,0.014525
5,Barbados,288071,13.193887,-59.543198,22037.0,0.076499
6,Belgium,11669155,50.503887,4.469936,428729.0,0.036740
7,Brazil,215277900,-14.235004,-51.925280,6002873.0,0.027884
8,Bulgaria,6892210,42.733883,25.485830,118549.0,0.017200
9,Cambodia,16943711,12.565679,104.990963,7770.0,0.000459


In [33]:
# Calculate the total percent vaccinated in the world
percentageTotal = vaccinationAndPopulationByLocation["people_vaccinated"].sum() / vaccinationAndPopulationByLocation["Population"].sum()
print('Percentage Vaccinated in the world: {}%'.format(round(percentageTotal*100, 2))) 

Percentage Vaccinated in the world: 3.57%


In [29]:
# Create the Marker map
marker_map = folium.Map(location=[0,0], tiles='cartodbpositron', zoom_start=2) 

# Add points to the map
mc = MarkerCluster()
for idx, row in vaccinationAndPopulationByLocation.iterrows(): 
    if not math.isnan(row['longitude']) and not math.isnan(row['latitude']):
        mc.add_child(Marker(location=[row['latitude'], row['longitude']],
                            tooltip=str(round(row['percent_vaccinated']*100, 2))+"%"))
marker_map.add_child(mc)

marker_map

In [32]:
# Create the Heatmap
heat_map = folium.Map(location=[0,0], tiles='cartodbpositron', zoom_start=2) 

# Add a heatmap to the base map
HeatMap(data=vaccinationAndPopulationByLocation[['latitude', 'longitude']], radius=10).add_to(heat_map)

heat_map